# Introduction to NLP Fundematals in TensorFlow
NLP has the goal of deriving information out of natural language(could be sequences text or speech)
Another common term for NLP problems is sequence to squence problems(seq2seq)

In [ ]:
!nvidia-smi -L

## Get helper functions

In [31]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-10-31 12:51:37--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-10-31 12:51:37 (41.6 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [32]:
from helper_functions import create_tensorboard_callback,unzip_data,plot_loss_curves,compare_historys

## Get a text dataset
the dataset we're going to be using is Kaggle's introduction to NLP dataset(text samples of tweets labelled as diaster or not diaster).

In [33]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-10-31 12:51:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.207, 173.194.69.207, 142.250.153.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.19MB/s    in 0.5s    

2025-10-31 12:51:46 (1.19 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [35]:
# unzip the data
unzip_data("nlp_getting_started.zip")

In [36]:
import tensorflow as tf
import pandas as pd


In [37]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [38]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [39]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [40]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [41]:
# what does the tesst dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [42]:
# how many examples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [ ]:
len(train_df),len(test_df)

In [43]:
# Let's visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row 
    print(f"target: {target}","(real diaster)" if target > 0 else "(not real diaster)" )
    print(f"Text:\n{text}\n")
    print(10*"__","\n")

target: 1 (real diaster)
Text:
ÛÏ@FDNY: This morning #FDNY responded to a sinkhole in #Brooklyn. Units remain on-scene with @NYCBuildings &amp; others. http://t.co/M78ir0IK01Û

____________________ 

target: 1 (real diaster)
Text:
Top insurer blasts lack of Australian Govt action on disaster mitigation http://t.co/sDgOUtWNtb via @smh

____________________ 

target: 0 (not real diaster)
Text:
Mere sight of a gun makes police ÛÒ and public ÛÒ more aggressive experts say http://t.co/N4NEUIyt2k

____________________ 

target: 1 (real diaster)
Text:
Armageddon https://t.co/uCSUDk3q1d

____________________ 

target: 0 (not real diaster)
Text:
That moth that held me hostage yesterday has been chilling on the bathroom windowsill all day and I'm not okay with this

____________________ 



### Split data into training and validation datasets

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [46]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [47]:
# check the first ten samples
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into number, machine learning don't know text.
when dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.
There are a few ways to do this, namely:
* *Tokenization* - direct mapping of token (a token could be a word, a character or in between) to number.
* *Embedding* - Create a matrix of feature vector for each token ( the size of the feature vector can be defined and this embedding can be learned)

### Text vecorization(tokenization)

In [48]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [49]:
text_vectorizer = TextVectorization(max_tokens=None, # how many word in the vocablary (automatically add <OOV>
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map token to number
                                    output_sequence_length=None ,# how long do you want your sequence to be
                                    )

In [50]:
# find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

In [51]:
# setup  text vectorization variables
max_vocab_length =10000 # max number of words to have in our vocablary
max_length = 15 # max lenght our sequences will be ( e.g how many words from a tweet does a model see?)


In [53]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length
                                   )

In [55]:
# Fit the text vectorizer to the training sentence
text_vectorizer.adapt(train_sentences)

In [56]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [57]:
# Choose a random sentece from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \n\nVectorized Version: {text_vectorizer(random_sentence)} ")
text_vectorizer(random_sentence)

Original text:
 Enugu Government to demolish illegal structures at International Conference Centre http://t.co/7K5SHaiqIw 

Vectorized Version: [1297  547    5  516 1836 2276   17 1199 1877 1036    1    0    0    0
    0] 


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([1297,  547,    5,  516, 1836, 2276,   17, 1199, 1877, 1036,    1,
          0,    0,    0,    0])>

In [58]:
# check if token is a sentence have the same int value across different sentences
sample_sentence_two = "schools are the best Western in Lit lit litterally.. LiTTErALLy.."
text_vectorizer(sample_sentence_two)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([2718,   22,    2,  149, 1102,    4, 5214, 5214,    1,    1,    0,
          0,    0,    0,    0])>

In [59]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in vocabulary
top_5_words = words_in_vocab[:5] # get the most common word
bottom_5_words = words_in_vocab[-5:] # get the least common word
top_5_words,bottom_5_words,len(words_in_vocab)

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'],
 10000)

### Creating and Embedding using an Embedding Layer
To make our embedding, we going to use tensorflow embedding layer
The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector,for example, a value of 100 would mean each token gets represented by a vector 100 long
*  `input_length` = length of the sequences being passed to the embedding layer

In [60]:
 from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                            output_dim=128,
                             input_length=max_length
                            )


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"original text: \n{random_sentence} ")
random_sentence_vectorized = text_vectorizer(random_sentence)
sample_embed = embedding(random_sentence_vectorized)
random_sentence_vectorized,sample_embed

original text: 
Investigators shift focus to cause of fatal Waimate fire http://t.co/aDSvDpNP3r 


(<tf.Tensor: shape=(15,), dtype=int64, numpy=
 array([ 420,  988, 2109,    5,  257,    6,  163, 2615,   42,    1,    0,
           0,    0,    0,    0])>,
 <tf.Tensor: shape=(15, 128), dtype=float32, numpy=
 array([[ 4.7201406e-02, -9.5920675e-03, -2.4193002e-02, ...,
         -9.9213608e-03, -4.3546535e-02,  1.7108891e-02],
        [-4.4974852e-02,  4.2800616e-02,  3.7815463e-02, ...,
          3.9716411e-02,  2.4020281e-02, -1.1399437e-02],
        [-2.7111148e-02, -1.8554293e-02, -4.5135714e-02, ...,
          4.5683395e-02,  5.2236021e-05,  8.9861751e-03],
        ...,
        [ 2.6647855e-02,  2.7680922e-02, -1.4325418e-02, ...,
         -2.2269094e-02,  1.0610722e-02,  2.9167797e-02],
        [ 2.6647855e-02,  2.7680922e-02, -1.4325418e-02, ...,
         -2.2269094e-02,  1.0610722e-02,  2.9167797e-02],
        [ 2.6647855e-02,  2.7680922e-02, -1.4325418e-02, ...,
         -2.2269094e-02,  1.0610722e-02,  2.9167797e-02]], dtype=float32)>)

In [62]:
# check out a single token's embedding
sample_embed[0], sample_embed[0].shape,random_sentence_vectorized[14]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04720141, -0.00959207, -0.024193  , -0.01261927, -0.01031259,
        -0.02490903,  0.01632405, -0.02368134,  0.01414777,  0.00334474,
         0.03338731,  0.01977942, -0.01160901,  0.04723107,  0.00597067,
         0.00070716, -0.00610645,  0.02543307,  0.04448214,  0.01200265,
         0.04859145,  0.01230508, -0.04115115, -0.02860166, -0.04703654,
        -0.03888854, -0.01194274,  0.00793676, -0.01742103,  0.0146677 ,
        -0.03307382, -0.00890846,  0.04887294, -0.04852685,  0.03917069,
        -0.02458229, -0.00862589,  0.0269776 ,  0.0083824 , -0.01838181,
         0.00584043, -0.0486443 , -0.03473888, -0.02024984,  0.00895911,
        -0.00814612,  0.01972857,  0.01198785, -0.00893375, -0.02231455,
         0.01002989,  0.01747633,  0.04375396, -0.04298608, -0.0422945 ,
         0.01861909, -0.01085585, -0.04459664, -0.01454009, -0.0486149 ,
        -0.047469  , -0.0344394 ,  0.04315026, -0.01709048, -0.0061091 ,
  

## Modeling a text dataset (running a series of experiment)
Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.
we'll start with a baseline and move on from there.

* Model 0: Naive Bayes(baseline)
* Model 1: Feed-Forwared neural Network(dense Model)
* Model 2: LSTM model(RNN)
* Model 3: GRU model(RNN)
* Model 4: Bidirectional-LSTM model(RNN)
* Model 5:1D Convolutional Neural Network(CNN)
* Model 6: TensorFlow Hub pretrained Feature Extractor(using transofer learning for NLP)
* Model 7: same as model 6 with 10% of training data

  How we are going to approach all of these?

  Use the standard steps in modelling with tensorflow:
  * Create a model
  * Build a model
  * Fit a model
  * Evaluate a model
  

### Model 0: Getting a baseline 
As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiment to build on

In [ ]:
# !pip install scikit-learn
import sklearn
print(sklearn.__version__)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
# Create tokenization and modelling pipeline 
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf",MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0_history = model_0.fit(train_sentences,train_labels)

In [ ]:
model_0_history

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
baseline_score

In [ ]:
print(f"Our baseline model achievs an accuracy of: {baseline_score}")
train_df.target.value_counts()

In [ ]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:2],val_sentences[:2]

### Creating an evaluation function for our model experiments
we could evaluate all of our model's predictions with different metrics every time,however, this will be cumbersome and could easily be fixed using function
Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Percision
* Recall
* F1-score

In [63]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true, y_preds):
    """
    Calculates model accuracy, recall, precision and f1-score
    of a binary classification model.
    """
    model_accuracy = accuracy_score(y_true, y_preds) * 100
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(
        y_true, y_preds, average="weighted"
    )
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1,
    }
    return model_results


In [ ]:
baseline_results = calculate_results(y_true=val_labels,y_preds=baseline_preds)

In [ ]:
baseline_results

### Model 1: A simple dense model

In [64]:
# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"
# Bild model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(),dtype=tf.string,name="input_layer")
x = text_vectorizer(inputs) # turn the input text into number
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D(name="globalAverage_layer")(x)
# x = layers.Flatten()(x)
outputs = layers.Dense(1,activation="sigmoid",name="output_layer")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [ ]:
model_1.summary()

In [ ]:
# compile the model
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [ ]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                             y=train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback("TensorBoard","model_1_dense")])

In [ ]:
plot_loss_curves(model_1_history)

In [ ]:
model_1.evaluate(val_sentences,val_labels)

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)

In [ ]:
model_1_pred_probs.shape,val_sentences[:5],model_1_pred_probs[:5]


In [ ]:
# convert model prediction probablity to a label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

In [ ]:
model_1_results = calculate_results(y_true=val_labels,y_preds=model_1_preds)

In [ ]:
model_1_results

In [ ]:
baseline_results

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

## Visualizing Learned Embeddings

In [ ]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

In [ ]:
# Model 1 summary
model_1.summary()

In [ ]:
# get the weight metrics of embedding layer
# (these are the numberical representations of each token in our training data, which have been learned for 5 epochs)
print(f"total parameter of embedding weights: {len(embedding.get_weights()[0])}")
embed_weights = model_1.get_layer("embedding_5").get_weights()[0]
print(embed_weights[:20])
print(f"shape of embedding: {embed_weights.shape} ")

Now we've got the embedding matrix our mdoel has learned to represent our tokens, let's see how we can visualize it.
to do so, TensorFlow as a handy tool called projector.
and tensorflow alos has an incredible guide on word embeddings themselves.

In [ ]:
words_in_vocab[:10]

In [ ]:
# Create embedding files from tensorflow word embedding docs
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
import tensorboard
print(tensorboard.__version__)

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/TensorBoard/model_1_dense

## Recurrent Neural Networks (RNN's)
RNN's are useful sequence data.

The promise of a recurrent neural network is to use the ***`representation`*** of a previous input to aid the representation of a later input.

### model 2: LSTM
LSTM = long short-term memory (one of the most popular LSTM cells)
OUr structure of an RNN typically looks like this:
```
Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Dense) -> Output(label probability)
```

In [ ]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
# print(x.shape)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

In [ ]:
# make prediction with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
val_labels[:10]

In [ ]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,y_preds=model_2_preds)
model_2_results

In [ ]:
baseline_results

In [ ]:
from math import e
print(e)
def tanh_func(z):
    a = (e**z - e**(-z))/(e**z + e**(-z)) # tensorflow don't know math and python label code durring computational graph, it must be tensorflow optrational for derivation of gradients
    return a
    # return (tf.exp(z) - tf.exp(-z)) / (tf.exp(z) + tf.exp(-z))
    # return tf.math.tanh(z)
tanh_func(1.)

### build GRU model 3
Another popular and effective RNN component is the GRU or gated recurrent unit.
The GRU cell has similar features to an LSTM cell but has less parameters.

In [ ]:
inputs = layers.Input(shape=(1,),dtype=tf.string,name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(units=64,activation=tanh_func)(x)
# x = layers.Lambda(tanh_func)(x)
outputs = layers.Dense(1,activation="sigmoid",name="output_layer")(x)
model_3 = tf.keras.Model(inputs,outputs)


In [ ]:
model_3.summary()

In [ ]:
# compile the model
model_3.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_GRU")])

In [ ]:
plot_loss_curves(model_3_history)

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
model_3_results = calculate_results(y_true=val_labels,y_preds=model_3_preds)
model_3_results

### Model 4: Bidirectional RNN
Normal RNN go from left to right (just like you'd read and English sentence), however, a bidirectional RNN goes from right to left as well as left to right.


In [ ]:
# Build a bidirectional RNN in tensorflow
import tensorflow as tf
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
print(x.shape)
x = embedding(x)
print(x.shape)
# x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
# print(x.shape)
x = layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs)


In [ ]:
model_4.summary()

In [ ]:
model_4.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

In [ ]:
plot_loss_curves(model_4_history)

In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
model_4_results = calculate_results(y_true=val_labels,y_preds=model_4_preds)
model_4_results

## Convolutional Neural Network for Text(and other types of sequences)
We've used CNNs for images but images are typically 2D (hieght X Widht)... however, ouor text data is 1D.
Previously we've Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for seqeunces (in our case,text):
```
Inputs(xtext) -> Toknization -> Embedding -> Layer(s) (typicaly Conv1D and some kind of Pooling layer) -> outputs(Class probability)
```


### Model 5: Conv1D


In [26]:
# Test out our embedding layer,Conv1D layer and max Pooling layer
vectorizer_test = text_vectorizer(["this is test sentences"])
embedding_test = embedding(vectorizer_test)
conv_1d = layers.Conv1D(filters=32,
                       kernel_size=5,
                       activation="relu",
                       padding="valid")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

NameError: name 'text_vectorizer' is not defined

In [ ]:
vectorizer_test

In [ ]:
voc_words = text_vectorizer.get_vocabulary()
voc_words[:10]

In [ ]:
voc_words[9]

In [ ]:
embedding_test

In [ ]:
conv_1d_output

In [ ]:
max_pool_output

In [ ]:
# create model 5
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string,name="input_layer") # make sure you pass dtype string in input layer otherwise it treat as integer and leads to graph excution error.
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32,
                 kernel_size=5,
                 activation="relu",
                 strides=1,
                 padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid",name="output_layer")(x)
model_5 = tf.keras.Model(inputs,outputs)


In [ ]:
model_5.summary()

In [ ]:
model_5.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [ ]:
# compile the model
model_5.fit(train_sentences,
               train_labels,
               epochs=5,
               validation_data=(val_sentences,val_labels),
               callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

In [ ]:
val_sentences[:10],val_labels[:10]

In [ ]:
# val_sentences = tf.constant(val_sentences, dtype=tf.string)
# val_labels = tf.constant(val_labels, dtype=tf.float32)
# # 
# model_5.evaluate(val_sentences, val_labels)


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

In [ ]:
model_5_results = calculate_results(y_true=val_labels,y_preds=model_5_preds)
model_5_results

## Model 6: Tensorflow Hub pretrained sentences Encoder
Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's universal sentence encoder

In [4]:
# # from huggingface_hub importing and using model
# from huggingface_hub import snapshot_download
# from tensorflow_hub import KerasLayer

# # Download the model locally from Hugging Face
# model_path = snapshot_download(repo_id="Dimitre/universal-sentence-encoder")

# # Load it as a Keras Layer
# model = KerasLayer(handle=model_path)

# # Generate embeddings
# embeddings = model([
#     "The quick brown fox jumps over the lazy dog.",
#     "I am a sentence for which I would like to get its embedding."
# ])

# print(embeddings)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

saved_model.pb:   0%|          | 0.00/8.22M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

variables.index:   0%|          | 0.00/3.19k [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

variables/variables.data-00000-of-00001:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.0508086  -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [15]:
# import kagglehub
# from tensorflow_hub import KerasLayer

# # Download latest version
# path = kagglehub.model_download("google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder")

# print("Path to model files:", path)
# model = KerasLayer(path)


Path to model files: /kaggle/input/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/2


In [70]:
import tensorflow_hub as hub

model = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                       trainable=False,
                       input_shape=[],
                       dtype=tf.string
                      )

embeddings = model([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding."
])

print(embeddings) # (2, 512)


tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.0508086  -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [75]:
embeded = model(["hello world"])
embeded.shape

TensorShape([1, 512])

In [76]:
text_vec_test = text_vectorizer("hello tensorflow this the embedding pretrained model")
text_vec_test

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([1400,    1,   19,    2,    1,    1, 3616,    0,    0,    0,    0,
          0,    0,    0,    0])>

In [77]:
embeded = embedding(text_vec_test)
embeded.shape,embeded[:20]

(TensorShape([15, 128]),
 <tf.Tensor: shape=(15, 128), dtype=float32, numpy=
 array([[ 0.04106346,  0.01319272,  0.01162927, ..., -0.02430424,
          0.00630056,  0.03895735],
        [ 0.02830743,  0.00486987,  0.03268256, ...,  0.0345865 ,
         -0.04999486, -0.03345146],
        [-0.02976241, -0.01092074,  0.03112445, ..., -0.00333148,
          0.04269382, -0.04233369],
        ...,
        [ 0.02664785,  0.02768092, -0.01432542, ..., -0.02226909,
          0.01061072,  0.0291678 ],
        [ 0.02664785,  0.02768092, -0.01432542, ..., -0.02226909,
          0.01061072,  0.0291678 ],
        [ 0.02664785,  0.02768092, -0.01432542, ..., -0.02226909,
          0.01061072,  0.0291678 ]], dtype=float32)>)

In [78]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(),dtype=tf.string,name="input_layer")
# x = text_vectorizer(inputs)
# x = embedding(x)
x = model(inputs)
outputs = layers.Dense(1,activation="sigmoid",name="output_layer")(x)
model_6 = tf.keras.Model(inputs,outputs)
model_6.summary()


TypeError: Exception encountered when calling layer 'keras_layer_11' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None,), dtype=string, sparse=False, name=input_layer>,) and kwargs: {} for signature: (inputs: TensorSpec(shape=<unknown>, dtype=tf.string, name=None)).

Call arguments received by layer 'keras_layer_11' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype

In [79]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# ✅ Rename to avoid conflict
use_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape=[], dtype=tf.string, trainable=False
)

# Define the actual model
inputs = layers.Input(shape=(), dtype=tf.string, name="input_layer")
x = use_layer(inputs)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)

model_6 = tf.keras.Model(inputs, outputs)
model_6.summary()


TypeError: Exception encountered when calling layer 'keras_layer_12' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None,), dtype=string, sparse=False, name=input_layer>,) and kwargs: {} for signature: (inputs: TensorSpec(shape=<unknown>, dtype=tf.string, name=None)).

Call arguments received by layer 'keras_layer_12' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype